In [1]:
import findspark
findspark.init()
findspark.find()

'C:\\Users\\HAL 9004\\.conda\\envs\\spark-polito\\lib\\site-packages\\pyspark'

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Excercise 36") \
    .getOrCreate()

In [3]:
spark

In [4]:
#Exercise 36
#
# Average value
#Input: a collection of (structured) textual csv files
#containing the daily value of PM10 for a set of sensors
#▪ Each line of the files has the following format
#sensorId,date,PM10 value (μg/m3)\n
#Output: compute the average PM10 value
#▪ Print the result on the standard output

## With RDD's

In [5]:
sc = spark.sparkContext

In [5]:
sc

<SparkContext master=local[*] appName=Excercise 35>

In [45]:
rdd_36 = sc.textFile("input_file_36.txt")

In [34]:
rdd_36.take(30)

['s1,2016-01-01,20.5',
 's2,2016-01-01,30.1',
 's1,2016-01-02,60.2',
 's2,2016-01-02,20.4',
 's1,2016-01-03,55.5',
 's2,2016-01-03,52.5']

In [46]:
rdd_36 = rdd_36.map(lambda a: a.split(","))

In [36]:
rdd_36.take(30)

[['s1', '2016-01-01', '20.5'],
 ['s2', '2016-01-01', '30.1'],
 ['s1', '2016-01-02', '60.2'],
 ['s2', '2016-01-02', '20.4'],
 ['s1', '2016-01-03', '55.5'],
 ['s2', '2016-01-03', '52.5']]

In [47]:
rdd_36 = rdd_36.map(lambda a: (1,float(a[2])) )
rdd_36.take(10)

[(1, 20.5), (1, 30.1), (1, 60.2), (1, 20.4), (1, 55.5), (1, 52.5)]

In [49]:
rdd_36_reduced = rdd_36.reduce(lambda a,b: (a[0]+b[0],a[1]+b[1] ))


In [52]:
rdd_36_reduced

(6, 239.20000000000002)

In [50]:
type (rdd_36_reduced)

tuple

In [53]:
rdd_36_reduced[1]/rdd_36_reduced[0]

39.86666666666667

## With DataFrames

In [65]:
rdd_36 = sc.textFile("input_file_36.txt")
rdd_36 = rdd_36.map(lambda a: a.split(","))
rdd_36.take(10)

[['s1', '2016-01-01', '20.5'],
 ['s2', '2016-01-01', '30.1'],
 ['s1', '2016-01-02', '60.2'],
 ['s2', '2016-01-02', '20.4'],
 ['s1', '2016-01-03', '55.5'],
 ['s2', '2016-01-03', '52.5']]

In [66]:
rdd_36 = rdd_36.map(lambda a: (a[0],a[1],float(a[2])) )
rdd_36.take(10)

[('s1', '2016-01-01', 20.5),
 ('s2', '2016-01-01', 30.1),
 ('s1', '2016-01-02', 60.2),
 ('s2', '2016-01-02', 20.4),
 ('s1', '2016-01-03', 55.5),
 ('s2', '2016-01-03', 52.5)]

In [84]:
df36 = rdd_36.toDF()

In [85]:
df36.show()

+---+----------+----+
| _1|        _2|  _3|
+---+----------+----+
| s1|2016-01-01|20.5|
| s2|2016-01-01|30.1|
| s1|2016-01-02|60.2|
| s2|2016-01-02|20.4|
| s1|2016-01-03|55.5|
| s2|2016-01-03|52.5|
+---+----------+----+



In [86]:
from pyspark.sql.functions import lit
df36 = df36.withColumn("_0", lit(0))


In [94]:
from pyspark.sql.functions import avg

df36.select(avg('_3'))

DataFrame[avg(_3): double]

In [95]:
df36.select(avg('_3')).show()

+-----------------+
|          avg(_3)|
+-----------------+
|39.86666666666667|
+-----------------+



## With SQL

In [97]:
rdd_36 = sc.textFile("input_file_36.txt")
rdd_36 = rdd_36.map(lambda a: a.split(","))
rdd_36 = rdd_36.map(lambda a: (a[0],a[1],float(a[2])) )
df_aux =rdd_36.toDF()



In [98]:
df_aux.createOrReplaceTempView("sql36")


In [99]:
sqlDF = spark.sql("SELECT * FROM sql36")
sqlDF.show()

+---+----------+----+
| _1|        _2|  _3|
+---+----------+----+
| s1|2016-01-01|20.5|
| s2|2016-01-01|30.1|
| s1|2016-01-02|60.2|
| s2|2016-01-02|20.4|
| s1|2016-01-03|55.5|
| s2|2016-01-03|52.5|
+---+----------+----+



In [100]:
sqlDF = spark.sql("SELECT AVG(_3) FROM sql36")
sqlDF.show()

+-----------------+
|          avg(_3)|
+-----------------+
|39.86666666666667|
+-----------------+



In [101]:
sqlDF

DataFrame[avg(_3): double]

In [105]:
spark.sql("SELECT * FROM sql36").show()

+---+----------+----+
| _1|        _2|  _3|
+---+----------+----+
| s1|2016-01-01|20.5|
| s2|2016-01-01|30.1|
| s1|2016-01-02|60.2|
| s2|2016-01-02|20.4|
| s1|2016-01-03|55.5|
| s2|2016-01-03|52.5|
+---+----------+----+



In [106]:
spark.sql("SELECT *, cast(_3 as float) FROM sql36").show()

+---+----------+----+----+
| _1|        _2|  _3|  _3|
+---+----------+----+----+
| s1|2016-01-01|20.5|20.5|
| s2|2016-01-01|30.1|30.1|
| s1|2016-01-02|60.2|60.2|
| s2|2016-01-02|20.4|20.4|
| s1|2016-01-03|55.5|55.5|
| s2|2016-01-03|52.5|52.5|
+---+----------+----+----+

